In [10]:
import json
import os
import shutil
from mmdet3d.apis import Base3DInferencer

In [12]:
mmdetection3d_root_dir = Base3DInferencer._get_repo_or_mim_dir("mmdet3d")
model_list = list(reversed(json.load(open('model_list.json'))['detection_3d']))

## Copy configs

In [ ]:
dst_dir = "detection3d_configs"
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)

# Copy base_configs to dst_dir/base
# Copy pre_trained_configs to dst_dir/pre_trained
os.makedirs(os.path.join(dst_dir, "base"))
os.makedirs(os.path.join(dst_dir, "pre_trained"))
for model in model_list:
    model_name = model['model_name']
    base_configs = model.get('base_configs', [])
    pre_trained_configs = model.get('pre_trained_configs', [])
    for config in base_configs:
        src = os.path.join(mmdetection3d_root_dir, config)
        dst = os.path.join(dst_dir, "base", os.path.basename(config))
        shutil.copy(src, dst)
    for pre_trained_config in pre_trained_configs:
        config = pre_trained_config['config']
        src = os.path.join(mmdetection3d_root_dir, config)
        dst = os.path.join(dst_dir, "pre_trained", os.path.basename(config))
        shutil.copy(src, dst)

In [27]:
from mmengine.config import Config, ConfigDict

# 1. Get model config
mmdetection3d_root_dir = "mmdetection3d"
model_list = list(reversed(json.load(open('model_list.json'))['detection_3d']))

model_item = model_list[0]
print(model_item)

model_name = model_item['model_name']
base_configs = model_item['base_configs']

is_pre_trained = False
config_path = f"{mmdetection3d_root_dir}/{base_configs[0]}"


# 2. Read parameters from config file
cfg = Config.fromfile(config_path)

{'name': 'CenterPoint', 'paper': 'CVPR', 'year': 2021, 'model_name': 'centerpoint', 'base_configs': ['configs/_base_/models/centerpoint_voxel01_second_secfpn_nus.py', 'configs/_base_/models/centerpoint_pillar02_second_secfpn_nus.py'], 'pre_trained_configs': [{'config': 'configs/centerpoint/centerpoint_voxel01_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d.py', 'weights': 'https://download.openmmlab.com/mmdetection3d/v1.0.0_models/centerpoint/centerpoint_01voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus/centerpoint_01voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220810_030004-9061688e.pth', 'metadata': {'Training Memory (GB)': 5.2}, 'results': [{'Task': '3D Object Detection', 'Dataset': 'nuScenes', 'Metrics': {'mAP': 56.11, 'NDS': 64.61}}]}, {'config': 'configs/centerpoint/centerpoint_voxel01_second_secfpn_head-dcn-circlenms_8xb4-cyclic-20e_nus-3d.py', 'weights': 'https://download.openmmlab.com/mmdetection3d/v1.0.0_models/centerpoint/centerpoint_01voxel_second_secfpn_dcn_circ

In [63]:
from src.config_factory.config_parameters import ConfigParameters
import re
p = ConfigParameters()
p.voxel_size = [0.55, 0.55, 0.15]

config_path = "/root/mmdetection3d-v1.x/detection3d_configs/pre_trained/pv_rcnn_8xb2-80e_kitti-3d-3class.py"
cfg = Config.fromfile(config_path)
text = cfg.pretty_text


text = re.sub("voxel_size\s*=\s*\[[0-9.]+,\s*[0-9.]+,\s*[0-9.]+\]", f"voxel_size={p.voxel_size}", text)
text = re.sub("voxel_size\s*=\s*\([0-9.]+,\s*[0-9.]+,\s*[0-9.]+\)", f"voxel_size={tuple(p.voxel_size)}", text)
text = re.sub("voxel_size\s*=\s*\[[0-9.]+,\s*[0-9.]+\]", f"voxel_size={p.voxel_size[:2]}", text)


FileNotFoundError: [Errno 2] No such file or directory: '/root/mmdetection3d-v1.x/detection3d_configs/pre_trained/../_base_/datasets/kitti-3d-3class.py'

In [58]:
print(text)

model = dict(
    data_preprocessor=dict(
        type='Det3DDataPreprocessor',
        voxel=True,
        voxel_layer=dict(
            max_num_points=20,
            max_voxels=(
                30000,
                40000,
            ),
            voxel_size=[
                0.2,
                0.2,
                8,
            ])),
    pts_backbone=dict(
        conv_cfg=dict(bias=False, type='Conv2d'),
        in_channels=64,
        layer_nums=[
            3,
            5,
            5,
        ],
        layer_strides=[
            2,
            2,
            2,
        ],
        norm_cfg=dict(eps=0.001, momentum=0.01, type='BN'),
        out_channels=[
            64,
            128,
            256,
        ],
        type='SECOND'),
    pts_bbox_head=dict(
        bbox_coder=dict(
            code_size=9,
            max_num=500,
            out_size_factor=4,
            post_center_range=[
                -61.2,
                -61.2,
                -10.0,
 

In [47]:
import re

lidar_dims = 3
# substitute in_channels
search_res = re.search("in_channels\s*=\s*[0-6],", text)
if search_res:
    text = re.sub("in_channels\s*=\s*[0-6],", f"in_channels={lidar_dims},", text, count=1)
else:
    raise ValueError("in_channels not found in config")

In [48]:
re.findall("in_channels\s*=\s*[0-6]", text)

['in_channels=3',
 'in_channels=2',
 'in_channels=5',
 'in_channels=1',
 'in_channels=1',
 'in_channels=4']